In [ ]:
import numpy as np
import cvxpy as cp
import scipy as sc
import pennylane as qml
import torch
import torch.nn as nn
import torch.nn.functional as Fn
import torch.optim as optim
from torch.autograd import Variable

In [ ]:
import torch
import torch.nn as nn

class neural_function(nn.Module):
    def __init__(self, dimension, hidden_layers):
        super(neural_function, self).__init__()

        self.dimension = dimension
        self.hidden_layers = hidden_layers

        # Create a list to hold the hidden layer modules
        self.hidden_layer_modules = nn.ModuleList()

        # Add the input layer
        self.hidden_layer_modules.append(nn.Linear(dimension, hidden_layers[0]))

        # Add the hidden layers
        for i in range(1, len(hidden_layers)):
            self.hidden_layer_modules.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))

        # Add the output layer
        self.lin_end = nn.Linear(hidden_layers[-1], 1)

    def forward(self, input):
        y = input.float()

        # Forward pass through each hidden layer
        for layer in self.hidden_layer_modules:
            y = torch.sigmoid(layer(y))

        # Forward pass through the output layer
        y = self.lin_end(y)

        return y

In [ ]:
def generate_random_density_matrix(dimension):
    # Generate a random complex Hermitian matrix
    rand_matrix = np.random.rand(dimension, dimension) + 1j * np.random.rand(dimension, dimension)
    hermitian_matrix = (rand_matrix + np.conj(rand_matrix.T)) / 2

    # Generate eigenvalues and eigenvectors of the Hermitian matrix
    eigenvalues, eigenvectors = np.linalg.eigh(hermitian_matrix)

    # Ensure eigenvalues are non-negative
    eigenvalues[eigenvalues < 0] = 0

    # Construct the density matrix
    density_matrix = np.dot(eigenvectors, np.dot(np.diag(eigenvalues), np.linalg.inv(eigenvectors)))

    # Normalize the density matrix
    density_matrix /= np.trace(density_matrix)

    return density_matrix

In [ ]:
# quantum circuit settings
num_wires = 4
num_layers = 10
num_shots = 1
num_of_samples = 100

# dimension of the system
N = 4

# initiate the quantum device
device = qml.device("default.mixed", wires=num_wires, shots=num_of_samples)

@qml.qnode(device)
def measure_rho(param):
    qml.QubitDensityMatrix(density_matrix, wires=[0, 1])
    qml.RandomLayers(param, wires=[0, 1])
    
    # measure in the computational basis
    result = qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))
    return result

In [ ]:
# Evaluate sandwiched renyi relative entropy using its analytical forumla
alpha = 2.5

# ignore the division by zero
np.seterr(divide = 'ignore')

sigma_a = np.array(sc.linalg.fractional_matrix_power(sigma, (1.0 - alpha) / (2 * alpha)))

Q = np.real(np.trace(sc.linalg.fractional_matrix_power(sigma_a @ rho_test @ sigma_a, alpha)))

np.seterr(divide = 'warn')

# print the Renyi Entropy
print(np.log(N) - (1.0 / (alpha - 1)) * np.log(Q))

In [ ]:
#@title Optimization using Gradient Descent without neural network.

# parameters of the optimization
num_of_epochs = 500
learning_rate = 0.08
num_of_samples = 100
deviation = 1
alpha = 2.5

# we store the eigenvalues in a matrix
W = deviation*np.random.rand(3,3)

param_init = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))


# intialize the cost function store
cost_func_store = []


# start the training
for epoch in range(1, num_of_epochs):


  # evaluate the gradient with respect to the quantum circuit parameters
    gradients = np.zeros_like((param_init))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # parameter-shift for the first term

      # forward evaluation
            forward_sum_1 = 0
            result = measure_rho(shifted)
            result = list(result)
            result[0] = list(map(int, result[0]))
            result[1] = list(map(int, result[1]))
            for sample in range(num_of_samples):
                forward_sum_1 += np.exp(((alpha-1)/alpha)*W[result[0][sample]][result[1][sample]])

      # normalize this sum
            forward_sum_1 = forward_sum_1/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # parameter-shift for the second term of both the terms of the objective function

      # backward evaluation
            backward_sum_1 = 0
            result = measure_rho(shifted)
            result = list(result)
            result[0] = list(map(int, result[0]))
            result[1] = list(map(int, result[1]))
            for sample in range(num_of_samples):
                backward_sum_1 += np.exp(((alpha-1)/alpha)*W[result[0][sample]][result[1][sample]])

      # normalize the backward sum
            backward_sum_1 = backward_sum_1/num_of_samples


      # parameter-shift rule
            gradients[i, j] = 0.5*alpha * (forward_sum_1 - backward_sum_1)

  # first copy the quantum circuit parameters before updating it
    prev_param_init = param_init.copy()

  # update the quantum circuit parameters
    param_init += learning_rate*gradients

    # evaluate the gradient with respect to the eigenvalues

    # 1 , 1
    E = np.zeros_like(W)
    E[1][1] = 1
    dW1_first_term = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW1_first_term += E[result[0][sample]][result[1][sample]]*np.exp(((alpha-1)/alpha)*W[result[0][sample]][result[1][sample]])

    # normalize it
    dW1_first_term = dW1_first_term/num_of_samples
    dW1_first_term -= (np.exp(W[1][1])/N)

    # 1 , -1
    E = np.zeros_like(W)
    E[1][-1] = 1
    dW2_first_term = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW2_first_term += E[result[0][sample]][result[1][sample]]*np.exp(((alpha-1)/alpha)*W[result[0][sample]][result[1][sample]])

    # normalize it
    dW2_first_term = dW2_first_term/num_of_samples
    dW2_first_term -= (np.exp(W[1][-1])/N)

    # -1 , 1
    E = np.zeros_like(W)
    E[-1][1] = 1
    dW3_first_term = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW3_first_term += E[result[0][sample]][result[1][sample]]*np.exp(((alpha-1)/alpha)*W[result[0][sample]][result[1][sample]])

    # normalize it
    dW3_first_term = dW3_first_term/num_of_samples
    dW3_first_term -= (np.exp(W[-1][1])/N)

    # -1 , -1
    E = np.zeros_like(W)
    E[-1][-1] = 1
    dW4_first_term = 0
    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        dW4_first_term += E[result[0][sample]][result[1][sample]]*np.exp(((alpha-1)/alpha)*W[result[0][sample]][result[1][sample]])

    # normalize it
    dW4_first_term = dW4_first_term/num_of_samples
    dW4_first_term -= (np.exp(W[-1][-1])/N)

    # update the eigenvalues
    W[1][1] += learning_rate*(alpha-1)*(dW1_first_term)
    W[1][-1] += learning_rate*(alpha-1)*(dW2_first_term)
    W[-1][1] += learning_rate*(alpha-1)*(dW3_first_term)
    W[-1][-1] += learning_rate*(alpha-1)*(dW4_first_term)

  # evaluate the cost function at these parameters
    first_term = 0
    result = measure_rho(param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        first_term += np.exp(((alpha-1)/alpha)*W[result[0][sample]][result[1][sample]])

  # normalize the cost sum
    first_term = first_term/num_of_samples

  # # Second term evaluation
    second_term = np.exp(W[1][1]) + np.exp(W[1][-1]) + np.exp(W[-1][1]) + np.exp(W[-1][-1])

  # normalize the second term sum
    second_term = second_term/N

    # add the cost function to the store
    cost_func_store.append(np.log(N) - (1/(alpha-1))*np.log(alpha*first_term + (1-alpha)*second_term))

  # print the cost
    print(np.log(N) - (1/(alpha-1))*np.log(alpha*first_term + (1-alpha)*second_term))

In [ ]:
#@title Optimization using Gradient Descent with neural network

# parameters of the optimization
num_of_epochs = 500
learning_rate = 0.03
num_of_samples = 100
dimension = 2
hidden_layer = [20]
alpha = 2.5

# initialize the neural network and quantum circuit parameters
neural_fn = neural_function(dimension, hidden_layer)
param_init = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))


# intialize the cost function store
cost_func_store = []


# start the training
for epoch in range(1, num_of_epochs):


  # evaluate the gradient with respect to the quantum circuit parameters
    gradients = np.zeros_like((param_init))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # parameter-shift for the first term

      # forward evaluation
            forward_sum_1 = 0
            result = measure_rho(shifted)
            result = list(result)
            result[0] = list(map(int, result[0]))
            result[1] = list(map(int, result[1]))
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[0][sample], result[1][sample]])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                forward_sum_1 += np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy())

      # normalize this sum
            forward_sum_1 = forward_sum_1/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # parameter-shift for the second term of both the terms of the objective function

      # backward evaluation
            backward_sum_1 = 0
            result = measure_rho(shifted)
            result = list(result)
            result[0] = list(map(int, result[0]))
            result[1] = list(map(int, result[1]))
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[0][sample], result[1][sample]])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                backward_sum_1 += np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy())

      # normalize the backward sum
            backward_sum_1 = backward_sum_1/num_of_samples



      # parameter-shift rule
            gradients[i, j] = 0.5*alpha * (forward_sum_1 - backward_sum_1)

  # first copy the quantum circuit parameters before updating it
    prev_param_init = param_init.copy()

  # update the quantum circuit parameters
    param_init += learning_rate*gradients

  # evaluate the gradient with respect to the neural network parameters

    # evaluate the first term
    grad_w1 = []
    grad_b1 = []
    for layer_index in range(len(hidden_layer)):
        grad_w1.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].weight))
        grad_b1.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].bias))
    grad_w2 = torch.zeros_like(neural_fn.lin_end.weight)
    grad_b2 = torch.zeros_like(neural_fn.lin_end.bias)

    result = measure_rho(prev_param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[0][sample], result[1][sample]])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        nn_result.backward()
        for layer_index in range(len(hidden_layer)):
            grad_w1[layer_index] += (np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy()))*neural_fn.hidden_layer_modules[layer_index].weight.grad*(1/num_of_samples)
            grad_b1[layer_index] += (np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy()))*neural_fn.hidden_layer_modules[layer_index].bias.grad*(1/num_of_samples)
        grad_w2 += (np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy()))*neural_fn.lin_end.weight.grad*(1/num_of_samples)
        grad_b2 += (np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy()))*neural_fn.lin_end.bias.grad*(1/num_of_samples)
        for layer_index in range(len(hidden_layer)):
            neural_fn.hidden_layer_modules[layer_index].weight.grad.zero_()
            neural_fn.hidden_layer_modules[layer_index].bias.grad.zero_()
        neural_fn.lin_end.weight.grad.zero_()
        neural_fn.lin_end.bias.grad.zero_()


  # evaluate the second term
    grad_w1_2 = []
    grad_b1_2 = []
    for layer_index in range(len(hidden_layer)):
        grad_w1_2.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].weight))
        grad_b1_2.append(torch.zeros_like(neural_fn.hidden_layer_modules[layer_index].bias))
    grad_w2_2 = torch.zeros_like(neural_fn.lin_end.weight.grad)
    grad_b2_2 = torch.zeros_like(neural_fn.lin_end.bias.grad)

    # result = measure_sigma(prev_param_init)
    for sample in range(num_of_samples):
        result = np.random.choice([-1, 1], size=2)
        nn_result = neural_fn(torch.from_numpy(result.flatten()))
        nn_result.backward()
        for layer_index in range(len(hidden_layer)):
            grad_w1_2[layer_index] += (np.exp(nn_result[0].detach().numpy()))*neural_fn.hidden_layer_modules[layer_index].weight.grad*(1/num_of_samples)
            grad_b1_2[layer_index] += (np.exp(nn_result[0].detach().numpy()))*neural_fn.hidden_layer_modules[layer_index].bias.grad*(1/num_of_samples)
        grad_w2_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin_end.weight.grad*(1/num_of_samples)
        grad_b2_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin_end.bias.grad*(1/num_of_samples)
        for layer_index in range(len(hidden_layer)):
            neural_fn.hidden_layer_modules[layer_index].weight.grad.zero_()
            neural_fn.hidden_layer_modules[layer_index].bias.grad.zero_()
        neural_fn.lin_end.weight.grad.zero_()
        neural_fn.lin_end.bias.grad.zero_()

  # evaluate the difference, i.e., the gradient
    nn_grad_W1 = []
    nn_grad_b1 = []
    for layer_index in range(len(hidden_layer)):
        nn_grad_W1.append(grad_w1[layer_index] - grad_w1_2[layer_index])
        nn_grad_b1.append(grad_b1[layer_index] - grad_b1_2[layer_index])
    nn_grad_W2 = grad_w2 - grad_w2_2
    nn_grad_b2 = grad_b2 - grad_b2_2

  # update the NN weights and normalize them
    with torch.no_grad():
        for layer_index in range(len(hidden_layer)):
            neural_fn.hidden_layer_modules[layer_index].weight += learning_rate*(alpha-1)*nn_grad_W1[layer_index]
            neural_fn.hidden_layer_modules[layer_index].bias += learning_rate*(alpha-1)*nn_grad_b1[layer_index]
        neural_fn.lin_end.weight += learning_rate*(alpha-1)*nn_grad_W2
        neural_fn.lin_end.bias += learning_rate*(alpha-1)*nn_grad_b2

  # evaluate the cost function at these parameters
    first_term = 0
    result = measure_rho(param_init)
    result = list(result)
    result[0] = list(map(int, result[0]))
    result[1] = list(map(int, result[1]))
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[0][sample], result[1][sample]])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        first_term += np.exp(((alpha-1)/alpha)*nn_result[0].detach().numpy())

  # normalize the cost sum
    first_term = first_term/num_of_samples

  # # Second term evaluation
    second_term = 0
    for sample in range(num_of_samples):
        result = np.random.choice([-1, 1], size=2)
        nn_result = neural_fn(torch.from_numpy(result.flatten()))
        second_term += np.exp(nn_result[0].detach().numpy())

  # normalize the second term sum
    second_term = second_term/num_of_samples

    # add the cost function to the store
    cost_func_store.append(np.log(N) -(1/(alpha-1))*np.log(alpha*first_term + (1-alpha)*second_term))

  # print the cost
    print(np.log(N) - (1/(alpha-1))*np.log(alpha*first_term + (1-alpha)*second_term))